In [1]:
import pandas as pd

# ------ 7 STEPS ETL PROCESS ---

# ----- STEP 1: prepair transactions (1 product in 1 purchase) to calculate purchases  -----
# (executing time: 30 seconds)


# read transactions from csv-file
transactions = pd.read_csv('transactions.csv')

# convert into date datatypes
transactions['Date'] = pd.to_datetime(transactions['Date'])

# remove dublicates
subset = ['Invoice ID', 'Line']
duplicat = transactions.duplicated(subset)
transactions_1= transactions.drop_duplicates(subset, keep='first')

# delete negative values in 'Line Total'
transactions_cleaned  = transactions_1[transactions_1['Line Total'] >= 0]

# choose sales only
sales_only = transactions_cleaned[transactions_cleaned['Transaction Type'] == 'Sale']  

In [2]:
# ----- STEP 2: calculating purchases -----
# (executing time: 1 min)


# calculating purchases
purchases = sales_only.groupby(['Invoice ID']).agg(
  # 6 purchase parametres
    customer_id = ('Customer ID', 'first'),
    date = ('Date', 'first'),
    store_id = ('Store ID', 'first'),
    employee_id = ('Employee ID', 'first'),
    payment_method = ('Payment Method', 'first'),
    currency = ('Currency', 'first'),
    transaction_type = ('Transaction Type', 'first' ),
  # 3 purchase aggregates
    purchases_sum = ('Line Total', 'sum'), 
    units_cnt = ('Line', 'count'),
    prods_cnt = ('Quantity', 'sum')
)

# setting index
purchases = purchases.reset_index().set_index('Invoice ID')

# adding purchase number for each customer column 
purchases['purchase_num'] = (
    purchases
    .sort_values(by=['customer_id', 'date']) 
    .groupby('customer_id')
    .cumcount() + 1
)

# calculating first purchase date for each customer
first_purchases = (
    purchases.groupby('customer_id', as_index=False)['date']
    .min()
    .rename(columns={'date': 'first_purchase_date'})
)

# convert index into column
purchases = purchases.reset_index()  

# adding first purchase date column
purchases_final = purchases.merge(
    first_purchases[['customer_id', 'first_purchase_date']],
    on='customer_id',
    how='left'
)

# writing to csv
purchases_final.to_csv('purchases_final.csv', index=False)

In [3]:
# ----- STEP 3: prepairing purchases to calculate customers -----
# (executing time: 1 min)


# reading transactions from csv-file calculated on Step-2 above 
purchases_final = pd.read_csv('purchases_final.csv')

# convert into date datatypes
purchases_final['date'] = pd.to_datetime(purchases_final['date'])
purchases_final['first_purchase_date'] = pd.to_datetime(purchases_final['first_purchase_date'])

# adding purchase value in EUR
exchange_rates = {
    'EUR': 1.0,
    'USD': 0.9139,
    'GBP': 1.1881,
    'CNY': 0.12645
}
purchases_final['total_eur'] = purchases_final.apply(
    lambda row: row['purchases_sum'] * exchange_rates[row['currency']],
    axis=1
)

In [4]:
# ----- STEP 4: calculating customers stats from purchases -----
# (executing time: 1 min)


# calculate first purchase info
first_purchase_info = (
    purchases_final[purchases_final['purchase_num'] == 1]
    .loc[:, ['customer_id', 'date', 'store_id', 'payment_method', 'currency', 'prods_cnt', 'total_eur']]
    .rename(columns={
       # 4 first purchase parametres
        'date': 'first_purchase_dt',
        'store_id': 'first_purchase_store',
        'payment_method': 'first_payment_method',
        'currency': 'first_currency',
       # 2 first purchase aggregates
        'prods_cnt': 'first_purchase_prods_cnt',
        'total_eur': 'first_purchase_sum'
    })
)

# choose repeated purchases for 6 first months only
followup_mask = (
    (purchases_final['purchase_num'] > 1) &
    (purchases_final['date'] <=purchases_final['first_purchase_date'] + pd.DateOffset(months=6))
)
followups = purchases_final[followup_mask]

# calculate repeated purchases aggregates
agg_followups = (
    followups.groupby('customer_id', as_index=False)
    .agg({
        'Invoice ID': 'count',
        'prods_cnt': 'sum',
        'total_eur': 'sum'
    })
    .rename(columns={
        'Invoice ID': 'next_purchases_cnt',
        'prods_cnt': 'next_prods_cnt',
        'total_eur': 'next_sum'
    })
)

# add repeated purchases aggregates to customers dataset
customer_stats = first_purchase_info.merge(agg_followups, on='customer_id', how='left')
customer_stats[['next_purchases_cnt', 'next_prods_cnt', 'next_sum']] = \
    customer_stats[['next_purchases_cnt', 'next_prods_cnt', 'next_sum']].fillna(0)

In [5]:
# ----- STEP 5: adding first purchase store parametres -----
# (executing time: 5 seconds)


# read stores info from csv-file
stores_1 = pd.read_csv('stores.csv')

# add first purchase store parametres
customer_stats = customer_stats.merge(
    stores_1[['Store ID', 'Country', 'City']],
    left_on='first_purchase_store',
    right_on='Store ID',
    how='left'
)
customer_stats.drop(columns=['Store ID'], inplace=True)

# rename first purchase store parametres
customer_stats.rename(columns={
    'City': 'store_city',  
    'Country': 'store_country' 
},
inplace=True                     
)

In [6]:
# ----- STEP 6: adding customers parametres -----
# (executing time: 10 seconds)


# read customers info from csv-file
customers_1 = pd.read_csv('customers.csv')

# calculate customers age
customers_for_merge = customers_1[[
    'Customer ID', 'Gender', 'Date Of Birth', 'City', 'Country', 'Job Title'
]].copy()
customers_for_merge['Date Of Birth'] = pd.to_datetime(customers_for_merge['Date Of Birth'], errors='coerce')
# Age for 18.03.2025
reference_date = pd.to_datetime('2025-03-18')
customers_for_merge['age'] = customers_for_merge['Date Of Birth'].apply(
    lambda dob: reference_date.year - dob.year - ((reference_date.month, reference_date.day) < (dob.month, dob.day))
)

# add customers parametres
customers_for_merge.rename(columns={
    'Customer ID': 'customer_id',
    'Gender': 'gender',
    'City': 'customer_city',
    'Country': 'customer_country',
    'Job Title': 'customer_job_title'
}, inplace=True)

customer_stats = customer_stats.merge(
    customers_for_merge,
    on='customer_id',
    how='left'
)

# drop curtomers bithdate column, we have customers age now
customer_stats.drop(columns=['Date Of Birth'], inplace=True)

C:\Users\710634\AppData\Local\Temp\ipykernel_34792\3714095353.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  customers_1 = pd.read_csv('customers.csv')


In [8]:
# ----- STEP 7: calculating additional customers characters and aggregates -----
# (executing time: 20 seconds)

# calculate cohort months
customer_stats['cohort_month'] = customer_stats['first_purchase_dt'].dt.strftime('%Y-%m')

# calculate age groups
bins = [-1, 17, 22, 27, 35, 50, 65, 120]  
labels = ['1: 0-17 years', '2: 18-22 years', '3: 22-27 years', '4: 28-35 years', 
          '5: 36-50 years', '6: 51-65 years', '7: 66+ years'] 
customer_stats['age_group'] = pd.cut(customer_stats['age'], 
                                     bins=bins, labels=labels, right=True)

# exclude outliers on first purchase value
customer_stats = customer_stats[
    (customer_stats['first_purchase_sum']<=1000) 
    & (customer_stats['first_purchase_prods_cnt']<51)
]

# calculate first purchase value groups
bins = [-1, 25, 35, 50, 90, 200, 949, 1000000]   
labels = ['1: 0-25 EUR', '2: 25-35 EUR', '3: 35-50 EUR', '4: 50-90 EUR', 
          '5: 90-200 EUR', '6: 200-949 EUR', '7: 949+ EUR']  
customer_stats['first_purchase_sum_group'] = pd.cut(customer_stats['first_purchase_sum'], 
                                                    bins=bins, labels=labels, right=True)

# calculate first purchase items number groups
bins = [0, 1, 2, 5, 14, 1000]  # 
labels = ['1: 1 item in first purchase', '2: 2 items in first purchase', 
          '3: 3-5 items in first purchase', '4: 6-14 items in first purchase', 
          '5: 15+ items in first purchase']  
customer_stats['first_purchase_prods_cnt_group'] = pd.cut(customer_stats['first_purchase_prods_cnt'], 
                                                          bins=bins, labels=labels, right=True)

# calculate returned customers (has repeated purchases)
customer_stats['returned_customer'] = customer_stats['next_purchases_cnt'] > 0

# delete cohorts with life time less than 6 months
customer_stats = customer_stats[customer_stats['first_purchase_dt']<'2024-09-01']

# write final dataset to csv-file
customer_stats.to_csv('customer_stats.csv', index=True)

In [9]:
# ----- FINAL CHECK -----
# checking that everything is good
# shoul be 1.1 M rows, 21 columns + zero column = customer_id
customer_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1136519 entries, 0 to 1280219
Data columns (total 22 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   customer_id                     1136519 non-null  int64         
 1   first_purchase_dt               1136519 non-null  datetime64[ns]
 2   first_purchase_store            1136519 non-null  int64         
 3   first_payment_method            1136519 non-null  object        
 4   first_currency                  1136519 non-null  object        
 5   first_purchase_prods_cnt        1136519 non-null  int64         
 6   first_purchase_sum              1136519 non-null  float64       
 7   next_purchases_cnt              1136519 non-null  float64       
 8   next_prods_cnt                  1136519 non-null  float64       
 9   next_sum                        1136519 non-null  float64       
 10  store_country                   1136519 non-nul